In [ ]:
import numpy as np
import scipy.io
from itertools import combinations
from scipy.special import comb

In [ ]:
def import_data(patient, d=3):
    '''...'''
    data = scipy.io.loadmat(f'/Users/anibal/Google Drive/Information cochains/O-info/OinfoSinfo/OinfoCopulasN{d+1}/Oinfop{patient}.mat')

    del data['__header__']
    del data['__version__']
    del data['__globals__']

    data['TC']  = (data['Sinfo'] + data['Oinfo'])/2
    data['DTC'] = (data['Sinfo'] - data['Oinfo'])/2
    
    return data

In [ ]:
def construct_boundary(d, r=19):
    '''...'''
    domain_basis = tuple(combinations(range(r+1), d+1))
    target_basis = tuple(combinations(range(r+1), d))
    target_basis_ix = {tuple(v): index for index, v in enumerate(target_basis)}
    N = comb(r+1, d+1, exact=True)
    M = comb(r+1, d, exact=True)
    D = scipy.sparse.csr_matrix((M, N), dtype=np.int8)
    for j in range(d+1):
        jth_faces_ix = [
                target_basis_ix[tuple(np.concatenate((l[:j], l[j+1:])))]
                for l in domain_basis
                ]
        D += scipy.sparse.csr_matrix ( 
                        ([(-1)**j]*N, (jth_faces_ix, range(N))), 
                        shape=(M, N), dtype=np.int8
                        )
    return D

def construct_coboundary(d, r=19):
    '''...'''
    return construct_boundary(d+1,r).T

In [ ]:
patients = [f'00{i}' for i in range(1,10)] \
         + [f'0{i}' for i in range(10,100)] \
         + [f'{i}' for i in range(100,165)]

In [ ]:
def construct_weights(d, inverse=False):
    num_rows = comb(20, d+1, exact=True)
    weights = np.zeros((num_rows, 1), dtype=np.float)
    for p in patients:
        data = import_data(p, d)
        weights += data['TC']
    weights /= 164
    
    if inverse:
        weights = np.reciprocal(weights)
    
    weights = np.reshape(weights, (num_rows,))
    
    return scipy.sparse.csr_matrix (( 
                         weights, (range(num_rows), range(num_rows))), 
                         shape=(num_rows, num_rows), dtype=float
                         )

## Laplacian

$L^{up}_i = W_i^{-1}\, B_{i+1}\, W_{i+1}\, B_i^T$

$L^{down}_i = B_{i-1}^T\, W_{i-1}^{-1}\, B_i\, W_i$

$L = L^{up} + L^{down}$

In [ ]:
print(' d=3', comb(20,4),
      '\n d=4', comb(20,5),
      '\n d=5', comb(20,6) )

In [ ]:
def construct_up_laplacian(d):
    BTa = construct_coboundary(d)
#     print('BTa', BTa.shape, type(BTa))

    Wp = construct_weights(d+1)
#     print('Wp', Wp.shape, type(Wp))

    Bp = construct_boundary(d+1)
#     print('Bp', Bp.shape, type(Bp))

    WIa = construct_weights(d, inverse=True)
#     print('WIa', WIa.shape, type(WIa))

    Lu = np.multiply(WIa, np.multiply(Bp ,np.multiply(Wp, BTa)))
#     print('Lu', Lu.shape, type(Lu))
    
    return Lu

def construct_down_laplacian(d):

    Wa = construct_weights(d)
#     print('Wa', Wa.shape, type(Wa))

    Ba = construct_boundary(d)
#     print('Ba', Ba.shape, type(Ba))

    WIm = construct_weights(d-1, inverse=True)
#     print('WIm', WIm.shape, type(WIm))

    BTm = construct_coboundary(d-1)
#     print('BTm', BTm.shape, type(BTm))

    Ld = np.multiply(BTm, np.multiply(WIm ,np.multiply(Ba, Wa)))
#     print('Ld', Ld.shape, type(Ld))
    
    return Ld

# Lu = construct_up_laplacian(4)
# Ld = construct_down_laplacian(4)

In [ ]:
def construct_laplacian(d, r=19, weights=None):
    '''...'''
    Ld = construct_down_laplacian(d)
    Lu = construct_up_laplacian(d)

    return Ld + Lu

# L = construct_laplacian(4)